# 笔记

## 概括(AI 生成+自己完善)

本教程主要阐述了如何在 PyTorch 中构建神经网络模型。核心内容包括：首先，强调了将模型和数据迁移到可用加速器（如 CUDA 或 CPU）的重要性。接着，详细演示了通过继承 `torch.nn.Module` 类来定义自定义神经网络结构的关键步骤：在 `__init__` 方法中初始化网络层（如 `nn.Flatten`、`nn.Linear`、`nn.ReLU` 以及 `nn.Sequential` 容器），并在 `forward` 方法中定义数据通过网络时的前向传播路径。教程还解释了如何实例化模型、打印模型结构以供检查，以及如何通过调用模型实例本身（而不是直接调用 `forward` 方法）来获取输出。此外，教程还逐层剖析了模型中各层（`nn.Flatten`，`nn.Linear`，`nn.ReLU`，`nn.Sequential`，`nn.Softmax`）的作用和数据在它们之间传递时的变化。最后，介绍了如何通过 `model.named_parameters()` 方法访问和查看模型中所有可学习参数的名称、大小和值。

## 关键函数、语法(自己总结+AI 优化)

- `print(model_instance)`: 当对一个继承自 `nn.Module` 的模型实例使用 `print()` 时，PyTorch 会递归地打印出该模型及其所有子模块的结构，包括每层的名称、类型和参数（如 `Linear` 层的输入输出特征数）。这在 Debug 时非常有用。
- `model_instance(input_data)` (调用模型): `nn.Module` 的实例是可直接调用的。当以输入数据为参数调用模型实例时 (例如 `output = model(input)`), 实际上会执行该模块的 `forward` 方法以及 PyTorch 在后台管理的一些钩子 (hooks)。**注意**：教程明确指出，**不应直接调用 `model.forward()`**。
- `torch.nn.Softmax(dim=None)`:
    -   一种激活函数，将输入张量（通常是神经网络最后一层的原始输出，称为 logits）转换为表示概率分布的张量。
    -   转换后的值都在 $[0, 1]$ 范围内，并且在指定的 `dim` 维度上总和为1。
    -   `dim`: 进行 Softmax 操作的维度。例如在多分类问题中，如果模型的输出是 `(batch_size, num_classes)`，则 `dim=1` 会把每个样本的类别分数转换为概率。
- `model_instance.named_parameters()`:
    -   `nn.Module` 的一个方法，返回一个迭代器，该迭代器产生模块的所有参数（包括其子模块的参数），同时提供参数的名称和参数张量本身。
    -   这对于按名称访问特定参数、检查参数属性或在优化器中为不同的参数组分别设置学习率等高级操作非常有用。
    -   与 `model.parameters()` (只返回参数张量) 相比，`named_parameters()` 还可以返回参数的名称，因此十分方便。

## 提问与解答(自己提问+AI 回答)

- **问题1 (关于教程中对模型调用的说明):**
  教程中强调 “Do not call `model.forward()` directly!”，而是应该通过 `model(X)` 的方式来获取模型输出。
    1.  为什么不推荐直接调用 `model.forward()`？直接调用它相较于调用 `model(X)` 会有什么问题？
    2.  `model(X)` 这种调用方式的背后有什么特性吗？`nn.Module` 是如何实现这一点的？

  - **解答:**
    1.  **为什么不直接调用 `model.forward()`:**
        直接调用 `model.forward(X)` 会绕过 PyTorch 在 `nn.Module` 的 `__call__` 方法中设置的许多重要钩子 (hooks) 和管理机制。这些钩子负责处理诸如：
        -   **注册和执行 forward/backward hooks**: 用户可以注册在 `forward` 或 `backward` 计算之前或之后执行的自定义函数。这些钩子在调试、分析或修改梯度时非常有用。直接调用 `forward()` 会跳过这些钩子的执行。
        -   **JIT (Just-In-Time) 编译相关处理**: 对于使用 TorchScript 进行 JIT 编译的模型，`__call__` 方法中可能包含与编译和优化相关的逻辑。
        -   **状态管理**: 一些模块（如 `BatchNorm` 或 `Dropout`）在训练和评估模式下行为不同。`__call__` 方法可能与这些状态的正确管理有关（尽管模式切换主要由 `.train()` 和 `.eval()` 控制）。
        -   **输入/输出处理**: `__call__` 可能还包含一些标准的输入转换或输出处理逻辑。

        虽然对于简单的前向传播，直接调用 `forward()` 可能得到相同的结果，但这样做会失去 PyTorch 框架提供的很多灵活性和强大功能，并且可能导致在更复杂的场景（如使用钩子、JIT 编译等）下出现意外行为或错误。因此，遵循推荐的 `model(X)` 调用方式是最佳实践。

    2.  **`model(X)` 背后的 Python 特性:**
        这种可调用性依赖于 Python 的 `__call__` 特殊方法。当你在一个类的实例后面加上括号并传递参数时 (如 `instance(arg1, arg2)`), Python 会自动调用该实例所属类的 `__call__` 方法。
        在 `torch.nn.Module` 的基类定义中，它实现了 `__call__` 方法。这个 `__call__` 方法内部会处理上述提到的各种钩子和管理机制，然后在其内部**正确地调用用户在子类中定义的 `forward` 方法**。
        因此，当你写 `output = model(X)` 时，实际上是执行了 `model.__call__(X)`，而 `nn.Module` 的 `__call__` 实现保证了你的 `forward` 方法会在正确的上下文中被执行。

    参考资料:
    -   PyTorch `nn.Module` 官方文档 (关于 `__call__` 和 `hooks` 的描述): [https://pytorch.org/docs/stable/generated/torch.nn.Module.html](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)

- **问题2 (关于 `nn.Sequential` 的使用):**
  教程中的 `NeuralNetwork` 类使用 `self.linear_relu_stack = nn.Sequential(...)` 来组织一系列线性层和激活函数。
    1.  使用 `nn.Sequential` 容器来构建网络层的优点和缺点有哪些？
    2.  相对于在 `forward` 方法中逐一定义并调用每一层 (例如 `x = self.layer1(x)`, `x = self.relu1(x)`, `x = self.layer2(x)`, ...)，`nn.Sequential` 在什么情况下适合使用，而在什么情况下可能不适合使用？
    3.  `nn.Sequential` 中的模块是如何命名的？如果需要访问 `nn.Sequential` 内部的特定子模块，应该如何操作？

  - **解答:**
    1.  **`nn.Sequential` 的优缺点:**
        -   **优点:**
            -   **简洁性**: 对于线性的、没有复杂分支或跳跃连接的网络结构，`nn.Sequential` 提供了一种非常简洁和易读的方式来定义模型。
            -   **方便性**: 无需在 `forward` 方法中显式地逐行调用每个层，减少了样板代码。
            -   **模块化**: 可以将一组有序操作封装成一个单一的模块。
        -   **缺点:**
            -   **灵活性受限**: `nn.Sequential` 假设数据严格按照定义的顺序依次通过所有模块。它不适合需要复杂数据流（如残差连接、多输入/多输出分支、注意力机制中的跳跃连接等）的网络架构。
            -   **可调试性/可访问性**: 如果需要对序列内部的特定层进行更复杂的操作或获取中间输出，可能不如显式定义每一层方便（尽管仍然可以访问）。

    2.  **适用场景:**
        -   **`nn.Sequential` 更优的情况**:
            -   构建简单的多层感知机 (MLP) 或卷积网络中的线性堆叠部分（例如，VGG 风格的卷积块）。
            -   当网络结构清晰，数据流是单向、无分支的时候。
            -   快速原型设计或定义模型的小组件。
        -   **`nn.Sequential` 可能不适用的情况 (显式定义更优):**
            -   **残差网络 (ResNets)**: 需要将输入添加到后续层的输出中。
            -   **Inception 网络 (GoogLeNet)**: 包含并行的卷积分支，并将它们的输出拼接起来。
            -   **U-Net 或其他具有跳跃连接的架构**: 需要将编码器部分的特征图传递给解码器部分的对应层。
            -   **循环神经网络 (RNNs) 或 Transformers 中的复杂交互**: 通常涉及更精细的控制和数据操作。
            -   当需要在 `forward` 方法中对中间层的输出进行特定操作（例如，获取某个中间激活用于辅助损失）时，显式定义可以使这些操作更清晰。

    3.  **模块命名与访问:**
        -   **命名**:
            -   如果在创建 `nn.Sequential` 时使用**位置参数**传递模块 (例如 `nn.Sequential(nn.Linear(10, 20), nn.ReLU())`)，模块会按照它们在序列中的位置自动命名为 "0", "1", "2", ...
            -   如果在创建 `nn.Sequential` 时使用 **`collections.OrderedDict`** 或**关键字参数** (从 PyTorch 1.3 开始，仅限 `OrderedDict`；较新版本对关键字参数的支持可能已改变，推荐查阅最新文档，但 `OrderedDict` 是经典方式)，可以为每个模块指定自定义名称。
                ```python
                from collections import OrderedDict
                seq_custom_names = nn.Sequential(OrderedDict([
                    ('fc1', nn.Linear(10, 20)),
                    ('relu1', nn.ReLU()),
                    ('fc2', nn.Linear(20, 5))
                ]))
                ```
        -   **访问子模块**:
            -   可以通过**索引**访问（如果使用默认数字命名）：`sequential_module[0]` 会返回第一个子模块。
            -   可以通过**属性访问**（如果使用 `OrderedDict` 定义了自定义名称）：`seq_custom_names.fc1` 或 `seq_custom_names[0]` (仍然可以通过索引)。
            -   也可以迭代 `sequential_module.children()` 或 `sequential_module.named_children()`。

    参考资料:
    -   PyTorch `nn.Sequential` 文档: [https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)

- **问题3 (关于 `model.named_parameters()` 的使用):**
  教程展示了使用 `model.named_parameters()` 来迭代打印模型参数的名称、大小和部分值。除了打印信息用于调试与理解代码以外，在实际的模型训练和部署中，`named_parameters()` 相对于 `parameters()` (只返回参数张量) 还有哪些更高级或更实用的应用场景？

  - **解答:**
    1.  **`named_parameters()` 的高级应用场景:**
        `named_parameters()` 提供了参数的名称，这在许多高级应用中至关重要，而不仅仅是迭代参数：
        -   **精细化的优化策略**: 可以基于参数名称为模型的不同部分设置不同的优化参数（如学习率、权重衰减）。
        -   **参数冻结/解冻**: 在迁移学习或分阶段训练中，可以根据参数名称选择性地冻结或解冻特定层的权重，控制哪些参数参与梯度更新。
        -   **模型分析与可视化**: 可以基于名称提取特定层的权重进行分析，例如可视化卷积核或查看特定参数的统计信息。
        -   **模型剪枝与量化**: 在模型压缩技术中，参数名称有助于识别和操作目标参数。
        -   **保存与加载特定参数**: 虽然通常保存整个 `state_dict`，但在某些特殊情况下，可能需要基于名称有选择地保存或加载参数子集。
        -   **自定义正则化**: 可以对特定名称模式的参数应用不同的正则化项。

    2.  **为不同层设置不同学习率:**
        优化器构造函数 (如 `torch.optim.SGD`, `torch.optim.Adam`) 通常接受一个参数列表或一个包含参数组 (parameter groups) 的列表。每个参数组可以有自己特定的优化选项 (如 `lr`, `weight_decay`)。
        `named_parameters()` 可以用来方便地将参数分组：
        ```python
        # 假设 model 有一个 'encoder' 子模块和一个 'decoder' 子模块
        optimizer = torch.optim.Adam([
            {'params': model.encoder.parameters(), 'lr': 1e-4}, # 编码器使用较小的学习率
            {'params': model.decoder.parameters(), 'lr': 1e-3}  # 解码器使用较大的学习率
        ])

        # 或者更细致地基于名称筛选：
        encoder_params = []
        decoder_params = []
        other_params = []
        for name, param in model.named_parameters():
            if 'encoder' in name:
                encoder_params.append(param)
            elif 'decoder' in name:
                decoder_params.append(param)
            else:
                other_params.append(param)
        
        optimizer = torch.optim.Adam([
            {'params': encoder_params, 'lr': 1e-4},
            {'params': decoder_params, 'lr': 1e-3},
            {'params': other_params, 'lr': 5e-4} # 其他参数
        ])
        ```

    3.  **参数冻结进行迁移学习:**
        在迁移学习中，通常会加载预训练模型的权重，然后可能冻结大部分预训练层，只训练最后几层或新添加的分类头。`param.requires_grad = False` 可以阻止参数被优化器更新。
        `named_parameters()` 可以帮助识别哪些参数需要冻结：
        ```python
        # 加载预训练模型
        # model = torchvision.models.resnet18(pretrained=True)
        
        # 冻结所有预训练参数
        for param in model.parameters():
            param.requires_grad = False
            
        # 假设我们替换了最后的全连接层
        # num_ftrs = model.fc.in_features
        # model.fc = nn.Linear(num_ftrs, num_classes) # 新的 fc 层参数默认 requires_grad=True

        # 或者，只冻结特定名称模式的层，例如所有 'conv' 或 'layer1' 的参数
        # for name, param in model.named_parameters():
        #     if 'fc' not in name: # 假设只训练名为 'fc' 的全连接层
        #         param.requires_grad = False
        #     else:
        #         param.requires_grad = True # 确保目标层是可训练的

        # 之后创建优化器时，它只会更新 requires_grad=True 的参数
        # optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # 这样会报错，因为没有可训练参数
        # optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
        # 或者更常见：
        # optimizer = torch.optim.SGD(model.fc.parameters(), lr=1e-3) # 如果只训练新的fc层
        ```
        通过检查参数名称，可以精确控制哪些层参与训练，这对于有效的迁移学习至关重要。

    参考资料:
    -   PyTorch 优化器文档 (关于 per-parameter options): [https://pytorch.org/docs/stable/optim.html](https://pytorch.org/docs/stable/optim.html)
    -   PyTorch 迁移学习教程 (通常会涉及参数冻结): 例如，[https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)


[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| **Build Model** \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Build the Neural Network
========================

Neural networks comprise of layers/modules that perform operations on
data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace
provides all the building blocks you need to build your own neural
network. Every module in PyTorch subclasses the
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules
(layers). This nested structure allows for building and managing complex
architectures easily.

In the following sections, we\'ll build a neural network to classify
images in the FashionMNIST dataset.


In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
=======================

We want to be able to train our model on an
[accelerator](https://pytorch.org/docs/stable/torch.html#accelerators)
such as CUDA, MPS, MTIA, or XPU. If the current accelerator is
available, we will use it. Otherwise, we use the CPU.


In [2]:
# device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

# 检查是否有可用的 CUDA (NVIDIA GPU)
if torch.cuda.is_available():
    device = "cuda"
# 如果没有，则使用 CPU
else:
    device = "cpu"
    
print(f"Using {device} device")

Using cuda device


Define the Class
================

We define our neural network by subclassing `nn.Module`, and initialize
the neural network layers in `__init__`. Every `nn.Module` subclass
implements the operations on input data in the `forward` method.


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`,
and print its structure.


In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model\'s
`forward`, along with some [background
operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0
corresponding to each output of 10 raw predicted values for each class,
and dim=1 corresponding to the individual values of each output. We get
the prediction probabilities by passing it through an instance of the
`nn.Softmax` module.


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


------------------------------------------------------------------------


Model Layers
============

Let\'s break down the layers in the FashionMNIST model. To illustrate
it, we will take a sample minibatch of 3 images of size 28x28 and see
what happens to it as we pass it through the network.


In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
==========

We initialize the
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784
pixel values ( the minibatch dimension (at dim=0) is maintained).


In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
=========

The [linear
layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its
stored weights and biases.


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
=======

Non-linear activations are what create the complex mappings between the
model\'s inputs and outputs. They are applied after linear
transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use
[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
between our linear layers, but there\'s other activations to introduce
non-linearity in your model.


In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1468,  0.0280, -0.7383, -0.6660,  0.3875, -0.0307,  0.0070, -0.3730,
         -0.3075, -0.0641, -0.1849, -0.4819,  0.0850, -0.0101, -0.1596,  0.5004,
         -0.1321, -0.2752,  0.1897, -0.5468],
        [-0.1795,  0.1484, -0.2716, -0.2378, -0.0269,  0.2804,  0.1326, -0.5016,
         -0.4034, -0.0127,  0.1885, -0.9075, -0.0886, -0.0461,  0.1952,  0.2138,
         -0.0021, -0.1152, -0.1247, -0.6948],
        [-0.0280,  0.1145, -0.2932, -0.3904,  0.3915,  0.0265,  0.1756, -0.0780,
         -0.0327, -0.1553, -0.0653, -0.7458,  0.0182,  0.0176, -0.0777,  0.3326,
         -0.0466,  0.1672,  0.2055, -0.5763]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0280, 0.0000, 0.0000, 0.3875, 0.0000, 0.0070, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0850, 0.0000, 0.0000, 0.5004, 0.0000, 0.0000,
         0.1897, 0.0000],
        [0.0000, 0.1484, 0.0000, 0.0000, 0.0000, 0.2804, 0.1326, 0.0000, 0.0000,
         0.0000, 0.1885, 0.0000, 0.0000, 0.0000, 0.19

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
is an ordered container of modules. The data is passed through all the
modules in the same order as defined. You can use sequential containers
to put together a quick network like `seq_modules`.


In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax
==========

The last linear layer of the neural network returns [logits]{.title-ref}
- raw values in \[-infty, infty\] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)
module. The logits are scaled to values \[0, 1\] representing the
model\'s predicted probabilities for each class. `dim` parameter
indicates the dimension along which the values must sum to 1.


In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
================

Many layers inside a neural network are *parameterized*, i.e. have
associated weights and biases that are optimized during training.
Subclassing `nn.Module` automatically tracks all fields defined inside
your model object, and makes all parameters accessible using your
model\'s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and
a preview of its values.


In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0177, -0.0082, -0.0012,  ...,  0.0116, -0.0172, -0.0034],
        [-0.0351,  0.0020, -0.0066,  ..., -0.0044,  0.0232, -0.0043]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0137,  0.0058], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0358, -0.0238,  0.0099,  ...,  0.0355,  0.0252,  0.0030],
        [-0.0029,  0.0276, -0.0078,  ..., -0.0023, -0.0205, -0.0151]],
       device='cuda:0', grad_fn=<Sl

------------------------------------------------------------------------


Further Reading
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)
